In [ ]:
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import datetime as dt

# find indices of grid cell containing x, y
def get_nearest_xy(x, y, x_array, y_array):
   """
     search for nearest grid cell in an array of grid cells and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
    """
   x_idx = (np.abs(x_array - x)).argmin()
   y_idx = (np.abs(y_array - y)).argmin()
   return x_idx, y_idx

my_home_dir_name = 'my_home_dir'
my_personal_bucket_name = 'my_bucket'
my_cmip6_bucket_name = 'cmip6_data'

loc = 'Addis Ababa'
lats_lons = [(9.25, 38.75)]

ssp = 'ssp245'
variable = 'tasmax'
variable_long = 'maximum_temperature'
units = 'C'
days_since = dt.datetime(1850, 1, 1)  # specific to time in nc file

season = 'annual'
months2include = {  # comment out undesired months
   'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June':6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

percentile = 0.5  # decimal
percentile_2 = 0.95  # optional 2nd percentile line, set to 0 otherwise

data_path = f'/home/{my_home_dir_name}/{my_cmip6_bucket_name}/ISIMIP_BASD_data/daily/{variable_long}/{ssp}'
save_path = f'/home/{my_home_dir_name}/{my_personal_bucket_name}/{loc}/distplot'

# read in model names from modelNames.txt
with open(f'/home/{my_home_dir_name}/modelNames.txt') as f:
    models = [line.strip() for line in f]

yrs_base = np.arange(2000, 2020 + 1)
yrs_fut = np.arange(2040, 2060 + 1)
yrs_fut2 = []  # np.arange(2070, 2090 + 1)

base_list = np.array([])
fut_list = np.array([])
fut2_list = np.array([])

# assemble df column names based on desired years
col1 = 'location'
col2 = '%s-%s' % (yrs_base[0], yrs_base[-1])
col3 = '%s-%s' % (yrs_fut[0], yrs_fut[-1])

if len(yrs_fut2) > 0:
    col4 = '%s-%s' % (yrs_fut2[0], yrs_fut2[-1])

In [ ]:
# open single file to get correct lat/lon indices
file = Dataset(f'{data_path}/{variable}_{models[0]}_{ssp}_basd_0.5deg_{yrs_base[0]}.nc', 'r')

lons_input = file.variables['lon'][:]
lats_input = file.variables['lat'][:]

lats_lons_idc = []
for lat, lon in lats_lons:
    lon_idx, lat_idx = get_nearest_xy(lon, lat, lons_input, lats_input)
    lats_lons_idc.append((lat_idx, lon_idx))
    
for lat, lon in lats_lons_idc:
    print(lats_input[lat], lons_input[lon])

In [ ]:
# read in data and build df based on years
df1 = pd.DataFrame(columns=[col1, col2])
df2 = pd.DataFrame(columns=[col1, col3])

if len(yrs_fut2) > 0:
    df3 = pd.DataFrame(columns=[col1, col4])

for year in np.arange(1971, 2100 + 1):
    for m in models:
        if year in yrs_base or year in yrs_fut or year in yrs_fut2:
            print(year, m)                
            file = Dataset(f'{data_path}/{variable}_{m}_{ssp}_basd_0.5deg_{year}.nc', 'r')
            time = pd.DataFrame([(days_since + dt.timedelta(i)) for i in list(file.variables['time'][:])])
            months_idc = time.index[time[0].dt.month.isin(list(months2include.values()))]

            # drop last index if leap year and February in months2include for easier merging of dfs
            if 'February' in months2include.keys() and ((year % 400 == 0) or (year % 4 == 0 and year % 100 != 0)):
                months_idc = months_idc[0:-1]
            
            for lat_lon in lats_lons_idc:
                data = file.variables[variable][months_idc, lat_lon[0], lat_lon[1]]

                df_loc = pd.DataFrame(columns=[col1])
                
                if year in yrs_base:
                    df_loc[col2] = data
                    df_loc[col1] = loc
                    df1 = pd.concat([df1, df_loc])
                elif year in yrs_fut:
                    df_loc[col3] = data
                    df_loc[col1] = loc
                    df2 = pd.concat([df2, df_loc])
                elif year in yrs_fut2:
                    df_loc[col4] = data
                    df_loc[col1] = loc
                    df3 = pd.concat([df3, df_loc])
                               
            file.close()
            
        else:
            pass

df = df1.combine_first(df2)

if len(yrs_fut2) > 0:
    df = df.combine_first(df3)

df.reset_index(drop=True)

location = df[col1]  # 'combine_first' moved location to end. This sequence brings it back to the first column
df = df.drop(columns=col1)
df.insert(loc=0, column=col1, value=location)

# save data
df.to_csv(f'{save_path}/{loc.replace(" ", "_")}_{variable}_distribution_data_{ssp}_{season}.csv', index=False)

df

In [ ]:
# load data
df = pd.read_csv(f'{save_path}/{loc.replace(" ", "_")}_{variable}_distribution_data_{ssp}_{season}.csv', header=0)

df.iloc[:, 1:] = df.iloc[:, 1:] - 273.15  # K to C

if units == 'F':
    df.iloc[:, 1:] = df.iloc[:, 1:] * 1.8 + 32  # C to F
    
df

In [ ]:
# plot distribution
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
plt.rcParams['font.family'] = ['Arial']

fig, ax = plt.subplots(figsize=(16,6))

# adjust these as needed for pretty plot
percentile_text_horz_offset, percentile_text_vert_offset = 1.5, 0.01  # adjusts Nth percentile text up and down
percentile2_text_horz_offset, percentile2_text_vert_offset = 0.9, 0.03  # adjusts (optional) 2nd Nth percentile text up and down
arrow_style = 'arc3,rad=0.1'  # adjusts curve of arrow associated with Nth percentile text
arrow2_style = 'arc3,rad=0.3'  # adjusts curve of arrow associated with (optional) 2nd Nth percentile text
legend_horz_adjust, legend_vert_adjust = 0.22, 0.92  # adjusts legend box from ll corner
plot_suptitle = False  # set to True to add '{loc} Daily Maximum Temperatures' title at top of plot, otherwise set to False
subplot_label = '(c)'  # (a), (b), (1), (2) etc. (use when making multiple plots for reports), set to '' if only one plot
label_horz_adjust, label_vert_adjust = 0.1, 0.004  # adjusts location of plot label text
plot_ssp = True  # adds ssp text to plot when set to True, otherwise set to False
ssp_horz_adjust, ssp_vert_adjust = label_horz_adjust + 0.18, label_vert_adjust + 0.13  # adjusts location of plot ssp text
ax.set_ylim((0, 0.2))  # adjust height of plot by altering second value - or comment out for default height near top of curve

# get Nth percentile values
percentiles = df.quantile(q=percentile, axis=0, numeric_only=True, interpolation='linear')
percentiles = np.round_(percentiles, decimals=1)
height_of_fut_perc_line = []

if percentile_2 != 0:
    percentiles_2 = df.quantile(q=percentile_2, axis=0, numeric_only=True, interpolation='linear')
    percentiles_2 = np.round_(percentiles_2, decimals=1)
    height_of_fut_perc2_line = []
    
fontsize = 20

if len(yrs_fut2) > 0:
    colors = ['#52bbaf', '#e9c46a', '#ebad7b']
else:
    colors = ['#52bbaf', '#ebad7b']    
   
# round xticks to nearest N, using min/max values of data as guiding limits
round_to = 5
xtick_min, xtick_max = np.around(np.min(df)[1] / round_to, decimals=0) * round_to, np.around(np.max(df)[-1] / round_to, decimals=0) * round_to

for col_i in np.arange(0, len(df.columns) - 1):
    dis = sns.kdeplot(df.iloc[:, col_i + 1], color=colors[col_i])
    dis_x = dis.lines[col_i].get_xydata()[:, 0]
    dis_y = dis.lines[col_i].get_xydata()[:, 1]
    dis.fill_between(dis_x, dis_y, color=colors[col_i], alpha=0.5)

    # find index where line is closest to percentile value
    percentile_idx = (np.abs(percentiles[col_i] - dis_x)).argmin()
    if percentile_2 != 0:
        percentile2_idx = (np.abs(percentiles_2[col_i] - dis_x)).argmin()

    if col_i == (len(df.columns) - 2): # save heights of last time period percentile line 
        height_of_fut_perc_line.append(dis_y[percentile_idx])
        if percentile_2 != 0:
            height_of_fut_perc2_line.append(dis_y[percentile2_idx])

    # 95th percentile line
    ax.vlines(x=percentiles[col_i], ymin=0, ymax=dis_y[percentile_idx], colors='k', ls='--', lw=1, alpha=0.5)
    if percentile_2 != 0:
        ax.vlines(x=percentiles_2[col_i], ymin=0, ymax=dis_y[percentile2_idx], colors='k', ls='--', lw=1, alpha=0.5)

ax.annotate('%d$^{th}$ percentile increases\nfrom %.1f to %.1f \u00B0%s' % \
            (percentile*100, percentiles[0], percentiles[-1], units), 
            xy=(percentiles[-1], height_of_fut_perc_line[0]),
            xytext=(percentiles[-1] + percentile_text_horz_offset, 
                    height_of_fut_perc_line[0] + percentile_text_vert_offset), 
            fontsize=fontsize-2, 
            arrowprops=dict(facecolor='black',
                            shrink=0.1,
                            alpha=0.3,
                            width=2,
                            connectionstyle=arrow_style))

if percentile_2 != 0:
    ax.annotate('%d$^{th}$ percentile increases\nfrom %.1f to %.1f \u00B0%s' % \
                (percentile_2*100, percentiles_2[0], percentiles_2[-1], units), 
                xy=(percentiles_2[-1], height_of_fut_perc2_line[0]),
                xytext=(percentiles_2[-1] + percentile2_text_horz_offset, 
                        height_of_fut_perc2_line[0] + percentile2_text_vert_offset), 
                fontsize=fontsize-2, 
                arrowprops=dict(facecolor='black',
                                shrink=0.1,
                                alpha=0.3,
                                width=2,
                                connectionstyle=arrow2_style))

ax.set_xlabel(f'Temperature (\u00B0{units})', fontsize=fontsize + 4)
#ax.set_xlabel('')
ax.set_ylabel('')

ax.set_xlim((xtick_min, xtick_max))
plt.xticks(np.arange(xtick_min, xtick_max, round_to), fontsize=fontsize)
ax.set_yticks([])

# build legend manually
legend_elements = []
for i in np.arange(len(colors)):
    legend_label = '%s\u2013%s' % (df.columns[i + 1][:4], df.columns[i + 1][-4:])
    legend_elements.append(Line2D([0], [0], color=colors[i], lw=2, label=legend_label))  

if plot_suptitle == True:
    plt.suptitle(f'{loc} Daily Maximum Temperatures', fontsize=fontsize+5, weight='bold')

if len(months2include) == 12:
    ax.set_title(f'{subplot_label} Annual', fontsize=fontsize+4, loc='left')  # , style='italic')
else:
    ax.set_title('{} {}, {}, {}'.format(subplot_label, *list(months2include.keys())), fontsize=fontsize+4, loc='left')  # , style='italic')
    
fig.legend(handles=legend_elements, bbox_to_anchor=(legend_horz_adjust, legend_vert_adjust), fontsize=fontsize+4) 

fig.subplots_adjust(wspace=0, hspace=0)
fig.tight_layout()

# add ssp
if plot_ssp == True:
    ax.annotate(ssp, xy=(xtick_min, 0), xytext=(xtick_min + ssp_horz_adjust, ssp_vert_adjust), fontsize=fontsize)  # , style='italic')

if percentile_2 != 0:
    plt.savefig(f'{save_path}/{loc.replace(" ", "_")}_{variable}_distribution_plot_{ssp}_{season}_{round(percentile*100)}th_and_{round(percentile_2*100)}th_perc.png')
else:
    plt.savefig(f'{save_path}/{loc.replace(" ", "_")}_{variable}_distribution_plot_{ssp}_{season}_{round(percentile*100)}th_perc.png')